### 7.2 Exercise: Dimensionality Reduction and Feature Selection

### Part 1: PCA and Variance Threshold in a Linear Regression

In [277]:
# First I will import some needed libraries
import pandas as pd
from importlib import reload
import sys
import numpy as np
from imp import reload
import nltk
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 
%matplotlib inline
import seaborn as sns 
import re
import string
from nltk.stem import WordNetLemmatizer
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

### Step 1: Import the housing data as a data frame and ensure that the data is loaded properly.

In [278]:
# I will use pandas to pull the data to create a data frame to work from
train_Data = pd.read_csv('train.csv')
train_Data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


### Step 2: Drop the "Id" column and any features that are missing more than 40% of their values.

In [279]:
# As seen above in the dataframe we see that some variable have missing data in the some rows. 
# Which we will use pandas to read the data again and specify the data with NA values as missing.
train_Data = pd.read_csv('train.csv', encoding = 'ISO-8859-1', na_values = 'missing')
train_Data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [280]:
# First I will check and see what variables have NA values and what percentage of the variable is NA
NA_output = train_Data.isnull().sum()/len(train_Data)
NA_output

Id               0.000000
MSSubClass       0.000000
MSZoning         0.000000
LotFrontage      0.177397
LotArea          0.000000
                   ...   
MoSold           0.000000
YrSold           0.000000
SaleType         0.000000
SaleCondition    0.000000
SalePrice        0.000000
Length: 81, dtype: float64

In [281]:
# Next I will drop the Id variable and all the others that are above 40% NA.
updated_train_Data = train_Data.drop(['Id', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis = 1)
updated_train_Data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


### Step 3: For numerical columns, fill in any missing data with the median value

In [282]:
update_train_Data = updated_train_Data.fillna(updated_train_Data.median())
update_train_Data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [283]:
# Next I will check and see what variables still have NA values and what percentage of the variable is NA
NA_output = update_train_Data.isnull().sum()/len(update_train_Data)
NA_output

MSSubClass       0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
Street           0.0
                ... 
MoSold           0.0
YrSold           0.0
SaleType         0.0
SaleCondition    0.0
SalePrice        0.0
Length: 75, dtype: float64

### Step 4: For categorical columns, fill in any missing data with the most common value (mode).

In [284]:
# Next I will update my categorical columns with the mode of the column 
update_train_Data['MasVnrType'] = update_train_Data['MasVnrType'].fillna(update_train_Data['MasVnrType'].mode()[0])
update_train_Data['BsmtQual'] = update_train_Data['BsmtQual'].fillna(update_train_Data['BsmtQual'].mode()[0])
update_train_Data['BsmtCond'] = update_train_Data['BsmtCond'].fillna(update_train_Data['BsmtCond'].mode()[0])
update_train_Data['BsmtExposure'] = update_train_Data['BsmtExposure'].fillna(update_train_Data['BsmtExposure'].mode()[0])
update_train_Data['BsmtFinType1'] = update_train_Data['BsmtFinType1'].fillna(update_train_Data['BsmtFinType1'].mode()[0])
update_train_Data['BsmtFinType2'] = update_train_Data['BsmtFinType2'].fillna(update_train_Data['BsmtFinType2'].mode()[0])
update_train_Data['Electrical'] = update_train_Data['Electrical'].fillna(update_train_Data['Electrical'].mode()[0])
update_train_Data['GarageType'] = update_train_Data['GarageType'].fillna(update_train_Data['GarageType'].mode()[0])
update_train_Data['GarageFinish'] = update_train_Data['GarageFinish'].fillna(update_train_Data['GarageFinish'].mode()[0])
update_train_Data['GarageQual'] = update_train_Data['GarageQual'].fillna(update_train_Data['GarageQual'].mode()[0])
update_train_Data['GarageCond'] = update_train_Data['GarageCond'].fillna(update_train_Data['GarageCond'].mode()[0])

In [285]:
# Next we can see that some numerical features are actually really categories so we can change them.
update_train_Data = update_train_Data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                                               "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}})

In [286]:
# Next I will recode some categorical features so they are ordered numbered information
update_train_Data = update_train_Data.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3,"BLQ" : 4, "ALQ" : 5, "GLQ" : 6},"BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},"BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},"ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},"ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},"Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, "Min1" : 7, "Typ" : 8},"GarageCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},"GarageQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},"HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},"KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},"LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},"LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},"PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},"Street" : {"Grvl" : 1, "Pave" : 2},"Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}})

In [287]:
# Next I will transform numerical variables that are suposed to be categorical such as
# YrSold, MoSold, OverallCond, ,GarageCars and MSSubClass
update_train_Data['YrSold'] = update_train_Data['YrSold'].astype(str)
update_train_Data['MoSold'] = update_train_Data['MoSold'].astype(str)
update_train_Data['OverallCond'] = update_train_Data['OverallCond'].astype(str)
update_train_Data['MSSubClass'] = update_train_Data['MSSubClass'].apply(str)
update_train_Data['GarageCars'] = update_train_Data['GarageCars'].apply(str)

In [288]:
# Now I will check and make sure we have no NAs
Update_NA_output = update_train_Data.isnull().sum()/len(update_train_Data)
Update_NA_output

MSSubClass       0.0
MSZoning         0.0
LotFrontage      0.0
LotArea          0.0
Street           0.0
                ... 
MoSold           0.0
YrSold           0.0
SaleType         0.0
SaleCondition    0.0
SalePrice        0.0
Length: 75, dtype: float64

In [289]:
# Next I will find most important features relative to target
print("Most important features relative to Sales Price")
corr = update_train_Data.corr()
corr.sort_values(["SalePrice"], ascending = False, inplace = True)
print(corr.SalePrice)

Most important features relative to Sales Price
SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
ExterQual        0.682639
KitchenQual      0.659600
BsmtQual         0.650138
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.472614
Fireplaces       0.466929
GarageYrBlt      0.466754
HeatingQC        0.427649
BsmtFinSF1       0.386420
BsmtExposure     0.361863
LotFrontage      0.334771
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
BsmtFinType1     0.298771
HalfBath         0.284108
LotArea          0.263843
PavedDrive       0.231357
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
BsmtCond         0.159445
GarageQual       0.145904
GarageCond       0.114733
ScreenPorch      0.111447
Functional       0.107619
PoolArea         0.092404
3SsnPorch       

As seen above the NA values have been filled with either the median or mode depending on if the columns are numerical or categorical.

### Step 5: Convert the categorical columns to dummy variables

In [290]:
# First I will check and see the difference between the numerical features and 
# the categorical features
categorical_features = update_train_Data.select_dtypes(include=['object']).columns
print(categorical_features)
numerical_features = update_train_Data.select_dtypes(exclude = ["object"]).columns
print(numerical_features)

print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
feat_num = update_train_Data[numerical_features]
feat_cat = update_train_Data[categorical_features]

Index(['MSSubClass', 'MSZoning', 'LandContour', 'LotConfig', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallCond',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType',
       'GarageFinish', 'GarageCars', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition'],
      dtype='object')
Index(['LotFrontage', 'LotArea', 'Street', 'LotShape', 'Utilities',
       'LandSlope', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'HeatingQC', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageYrBlt', 'GarageArea

In [291]:
# get all categorical columns in the dataframe
feat_num.head(10)

,LotFrontage,LotArea,Street,LotShape,Utilities,LandSlope,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,...,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
0,65.0,8450,2,4,4,3,7,2003,2003,196.0,...,3,2,0,61,0,0,0,0,0,208500
1,80.0,9600,2,4,4,3,6,1976,1976,0.0,...,3,2,298,0,0,0,0,0,0,181500
2,68.0,11250,2,3,4,3,7,2001,2002,162.0,...,3,2,0,42,0,0,0,0,0,223500
3,60.0,9550,2,3,4,3,7,1915,1970,0.0,...,3,2,0,35,272,0,0,0,0,140000
4,84.0,14260,2,3,4,3,8,2000,2000,350.0,...,3,2,192,84,0,0,0,0,0,250000
5,85.0,14115,2,3,4,3,5,1993,1995,0.0,...,3,2,40,30,0,320,0,0,700,143000
6,75.0,10084,2,4,4,3,8,2004,2005,186.0,...,3,2,255,57,0,0,0,0,0,307000
7,69.0,10382,2,3,4,3,7,1973,1973,240.0,...,3,2,235,204,228,0,0,0,350,200000
8,51.0,6120,2,4,4,3,7,1931,1950,0.0,...,3,2,90,0,205,0,0,0,0,129900
9,50.0,7420,2,4,4,3,5,1939,1950,0.0,...,3,2,0,4,0,0,0,0,0,118000


In [292]:
feat_cat.head(10)

,MSSubClass,MSZoning,LandContour,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallCond,...,Heating,CentralAir,Electrical,GarageType,GarageFinish,GarageCars,MoSold,YrSold,SaleType,SaleCondition
0,SC60,RL,Lvl,Inside,CollgCr,Norm,Norm,1Fam,2Story,5,...,GasA,Y,SBrkr,Attchd,RFn,2,Feb,2008,WD,Normal
1,SC20,RL,Lvl,FR2,Veenker,Feedr,Norm,1Fam,1Story,8,...,GasA,Y,SBrkr,Attchd,RFn,2,May,2007,WD,Normal
2,SC60,RL,Lvl,Inside,CollgCr,Norm,Norm,1Fam,2Story,5,...,GasA,Y,SBrkr,Attchd,RFn,2,Sep,2008,WD,Normal
3,SC70,RL,Lvl,Corner,Crawfor,Norm,Norm,1Fam,2Story,5,...,GasA,Y,SBrkr,Detchd,Unf,3,Feb,2006,WD,Abnorml
4,SC60,RL,Lvl,FR2,NoRidge,Norm,Norm,1Fam,2Story,5,...,GasA,Y,SBrkr,Attchd,RFn,3,Dec,2008,WD,Normal
5,SC50,RL,Lvl,Inside,Mitchel,Norm,Norm,1Fam,1.5Fin,5,...,GasA,Y,SBrkr,Attchd,Unf,2,Oct,2009,WD,Normal
6,SC20,RL,Lvl,Inside,Somerst,Norm,Norm,1Fam,1Story,5,...,GasA,Y,SBrkr,Attchd,RFn,2,Aug,2007,WD,Normal
7,SC60,RL,Lvl,Corner,NWAmes,PosN,Norm,1Fam,2Story,6,...,GasA,Y,SBrkr,Attchd,RFn,2,Nov,2009,WD,Normal
8,SC50,RM,Lvl,Inside,OldTown,Artery,Norm,1Fam,1.5Fin,5,...,GasA,Y,FuseF,Detchd,Unf,2,Apr,2008,WD,Abnorml
9,SC190,RL,Lvl,Corner,BrkSide,Artery,Artery,2fmCon,1.5Unf,6,...,GasA,Y,SBrkr,Attchd,RFn,1,Jan,2008,WD,Normal


In [293]:
dummies = pd.get_dummies(update_train_Data[['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallCond', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'MoSold',
       'YrSold', 'SaleType', 'SaleCondition']], drop_first=True)
Train_df_dummies = pd.concat([update_train_Data.drop(['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallCond', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'MoSold',
       'YrSold', 'SaleType', 'SaleCondition'],axis=1), dummies],axis=1)
Train_df_dummies

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.0,8450,7,2003,2003,196.0,706,0,150,856,...,0,0,0,0,1,0,0,0,1,0
1,80.0,9600,6,1976,1976,0.0,978,0,284,1262,...,0,0,0,0,1,0,0,0,1,0
2,68.0,11250,7,2001,2002,162.0,486,0,434,920,...,0,0,0,0,1,0,0,0,1,0
3,60.0,9550,7,1915,1970,0.0,216,0,540,756,...,0,0,0,0,1,0,0,0,0,0
4,84.0,14260,8,2000,2000,350.0,655,0,490,1145,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,6,1999,2000,0.0,0,0,953,953,...,0,0,0,0,1,0,0,0,1,0
1456,85.0,13175,6,1978,1988,119.0,790,163,589,1542,...,0,0,0,0,1,0,0,0,1,0
1457,66.0,9042,7,1941,2006,0.0,275,0,877,1152,...,0,0,0,0,1,0,0,0,1,0
1458,68.0,9717,5,1950,1996,0.0,49,1029,0,1078,...,0,0,0,0,1,0,0,0,1,0


### Step 6: Split the data into a training and test set, where the SalePrice column is the target.

In [294]:
from sklearn.model_selection import train_test_split
# First I will split into training and test data:
x = Train_df_dummies['SalePrice'] 
y = Train_df_dummies['SaleCondition_Normal']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.2,random_state=0)

In [295]:
# Next I will reset the indices in the training and test sets to prevent pandas slicing warnings
x_train = x_train.reset_index(drop = True) 
x_test = x_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

### Step 7: Run a linear regression and report the R2-value and RMSE on the test set.

In [296]:
# First I will import the LinearRegression from the from sklearn.linear_model and create an object
from sklearn.linear_model import LinearRegression
clf = LinearRegression() 

In [297]:
# Next I will fit the X and y test variables 
clf.fit(x_test.values.reshape(-1,1),y_test)

LinearRegression()

In [298]:
# Next I will predict the output by passing the x_test variable.
clf.predict(x_test.values.reshape(-1,1))

array([0.77601034, 0.80493827, 0.81477711, 0.77969948, 0.82418819,
       0.82547151, 0.74080523, 0.80151606, 0.54313934, 0.7981794 ,
       0.77247007, 0.8032678 , 0.76558287, 0.80921603, 0.81084157,
       0.79980495, 0.78055503, 0.80895936, 0.79794413, 0.79552719,
       0.79082166, 0.80001884, 0.81477711, 0.78739945, 0.78269391,
       0.7899661 , 0.78590223, 0.8256854 , 0.72494415, 0.81114102,
       0.81477711, 0.77071618, 0.79509942, 0.75488847, 0.70248591,
       0.78697167, 0.74312463, 0.81370767, 0.74932738, 0.72280527,
       0.7578829 , 0.80622159, 0.79125371, 0.74205519, 0.68943874,
       0.81092713, 0.80836047, 0.80699159, 0.78804111, 0.82551429,
       0.68537487, 0.79509942, 0.79000888, 0.81905487, 0.74419407,
       0.80921603, 0.80536604, 0.75920901, 0.80237161, 0.81263823,
       0.80301327, 0.80408271, 0.80431799, 0.7846189 , 0.77905782,
       0.79488553, 0.80536604, 0.76579676, 0.80237161, 0.76558287,
       0.7811967 , 0.81135491, 0.82675484, 0.69414428, 0.81392

In [299]:
# Next I will make a prediction to use train the algorithm
y_pred = clf.predict(x_test.values.reshape(-1,1))

In [300]:
# Next I will test the accuracy of the model
clf.score(x_test.values.reshape(-1,1),y_test)

0.007468608055410164

### R2-value Metric

In [301]:
# Next I will calculate the r2 score by using my y_test data and the predicted data
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print('r2 score for perfect model is', r2)

r2 score for perfect model is 0.007468608055410164


###  RMSE - Root Mean Squared Error Metric

In [302]:
# Next I will calculate the RMSE, MSE, and MAE to compare how well the algorithm performs
from sklearn import metrics
print('RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE - Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))

RMSE - Root Mean Squared Error: 0.4098050352448076
MAE - Mean Absolute Error: 0.33588033382399596
MSE - Mean Squared Error: 0.16794016691199798


### Step 8: Fit and transform the training features with a PCA so that 90% of the variance is retained (see section 9.1 in the Machine Learning with Python Cookbook).

In [303]:
# First I will load needed libraries 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

In [333]:
# Next I will standardize the features matrix
features = StandardScaler().fit_transform(x_train.values.reshape(-1,1), y_train.values.reshape(-1,1))
features

array([[ 1.70779177],
       [-0.90878376],
       [-0.22059023],
       ...,
       [-0.87692295],
       [ 0.67788466],
       [ 0.51220844]])

In [332]:
# I will now create a PCA that will retain a 90% variance
pca = PCA(n_components=0.90, whiten=True)
pca

PCA(n_components=0.9, whiten=True)

In [331]:
# Next I will conduct the PCA
feature_pca = pca.fit_transform(features)
feature_pca

array([[ 1.70706054],
       [-0.90839464],
       [-0.22049578],
       ...,
       [-0.87654747],
       [ 0.67759441],
       [ 0.51198913]])

### Step 9: How many features are in the PCA-transformed matrix?

In [327]:
# Last I will show the results of the features and PCA features
print("Original number of features:", features.shape[1])
print("Reduced number of features:", feature_pca.shape[1])

Original number of features: 1
Reduced number of features: 1


### Step 10: Transform but DO NOT fit the test features with the same PCA

In [341]:
# Next I will transform the test features with the same PCA
x_test_transform_features = pca.transform(x_test.values.reshape(-1,1))
x_test_transform_features

array([[200538.09804787],
       [132943.05287686],
       [109952.9008756 ],
       [191917.79061922],
       [ 87962.32070048],
       [ 84963.60522205],
       [282800.86019569],
       [140939.62748599],
       [744681.01047564],
       [148736.2877299 ],
       [208810.55448102],
       [136846.38085794],
       [224903.6608819 ],
       [122947.33461544],
       [119148.9616761 ],
       [144937.91479056],
       [189918.64696694],
       [123547.07771113],
       [149286.05223428],
       [154933.63305198],
       [165928.92313954],
       [144438.12887749],
       [109952.9008756 ],
       [173925.49774867],
       [184920.78783623],
       [167928.06679182],
       [177423.99914017],
       [ 84463.81930898],
       [319862.98436537],
       [118449.2613978 ],
       [109952.9008756 ],
       [212908.7989682 ],
       [155933.20487812],
       [249892.95653545],
       [372340.50523782],
       [174925.06957481],
       [277381.18175435],
       [112451.83044095],
       [2628

In [340]:
# Next I will transform the test features with the same PCA
y_test_transform_features = pca.transform(y_test.values.reshape(-1,1))
y_test_transform_features

array([[-5.77676862e-17],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [-5.77676862e-17],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [-5.77676862e-17],
       [ 9.99571826e-01],
       [-5.77676862e-17],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [-5.77676862e-17],
       [-5.77676862e-17],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [-5.77676862e-17],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.99571826e-01],
       [ 9.9

### Step 11: Repeat step 7 with your PCA transformed data.

In [342]:
# First I will fit the X and y test variables 
clf.fit(x_test_transform_features, y_test_transform_features)

LinearRegression()

In [343]:
# Next I will predict the output by passing the x_test variable.
clf.predict(x_test_transform_features)

array([[0.77567807],
       [0.80459361],
       [0.81442825],
       [0.77936563],
       [0.82383529],
       [0.82511807],
       [0.74048804],
       [0.80117287],
       [0.54290679],
       [0.79783764],
       [0.77213931],
       [0.80292386],
       [0.76525507],
       [0.80886954],
       [0.81049439],
       [0.7994625 ],
       [0.78022082],
       [0.80861298],
       [0.79760247],
       [0.79518657],
       [0.79048305],
       [0.79967629],
       [0.81442825],
       [0.7870623 ],
       [0.78235878],
       [0.78962786],
       [0.78556573],
       [0.82533186],
       [0.72463375],
       [0.81079371],
       [0.81442825],
       [0.77038618],
       [0.79475898],
       [0.75456525],
       [0.70218512],
       [0.78663471],
       [0.74280644],
       [0.81335927],
       [0.74900654],
       [0.72249578],
       [0.7575584 ],
       [0.80587639],
       [0.79091492],
       [0.74173746],
       [0.68914354],
       [0.81057991],
       [0.80801435],
       [0.806

In [346]:
# Next I will make a prediction to use train the algorithm
y_pred_pca = clf.predict(x_test_transform_features)

In [345]:
# Next I will test the accuracy of the model
clf.score(x_test_transform_features, y_test_transform_features)

0.007468608055410164

### R2-value Metric

In [347]:
# Next I will calculate the r2 score by using my y_test data and the predicted data
from sklearn.metrics import r2_score
r2 = r2_score(y_test_transform_features, y_pred_pca)
print('r2 score for perfect model is', r2)

r2 score for perfect model is 0.007468608055410164


### RMSE - Root Mean Squared Error Metric

In [348]:
# Next I will calculate the RMSE, MSE, and MAE to compare how well the algorithm performs
from sklearn import metrics
print('RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_transform_features, y_pred_pca)))
print('MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test_transform_features, y_pred_pca))
print('MSE - Mean Squared Error:', metrics.mean_squared_error(y_test_transform_features, y_pred_pca))

RMSE - Root Mean Squared Error: 0.409629567441754
MAE - Mean Absolute Error: 0.33573651864556653
MSE - Mean Squared Error: 0.1677963825225185


### Step 12: Take your original training features (from step 6) and apply a min-max scaler to them.

In [351]:
from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
scaled = scaler.fit_transform(x_train.values.reshape(-1,1), y_train)
print(scaled)

[[0.38871407]
 [0.10359672]
 [0.17858631]
 ...
 [0.10706846]
 [0.27648938]
 [0.25843633]]


### Step 13: Find the min-max scaled features in your training set that have a variance above 0.1 (see Section 10.1 in the Machine Learning with Python Cookbook).

In [356]:
# First I will import needed libraries and create a threshholder
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold
thresholder = VarianceThreshold(threshold=.1)

In [359]:
# Next I will create a high variance feature matrix
feature_high_var = thresholder.fit_transform(x_train.values.reshape(-1,1))
feature_high_var

array([[314813],
       [109500],
       [163500],
       ...,
       [112000],
       [234000],
       [221000]])

### Step 14: Transform but DO NOT fit the test features with the same steps applied in steps 12 and 13.

#### Take your original training features (from step 6) and apply a min-max scaler to them.¶

In [371]:
# First I will transform my test data
trans_scaled = scaler.transform(x_test.values.reshape(-1,1), y_test)
print(trans_scaled)

[[0.19708555]
 [0.09734513]
 [0.06342183]
 [0.18436578]
 [0.03097345]
 [0.02654867]
 [0.31846903]
 [0.10914454]
 [1.        ]
 [0.12064897]
 [0.20929204]
 [0.10310472]
 [0.23303835]
 [0.08259587]
 [0.07699115]
 [0.11504425]
 [0.18141593]
 [0.08348083]
 [0.12146018]
 [0.12979351]
 [0.1460177 ]
 [0.11430678]
 [0.06342183]
 [0.15781711]
 [0.1740413 ]
 [0.14896755]
 [0.16297935]
 [0.02581121]
 [0.37315634]
 [0.0759587 ]
 [0.06342183]
 [0.21533923]
 [0.13126844]
 [0.2699115 ]
 [0.45058997]
 [0.15929204]
 [0.31047198]
 [0.06710914]
 [0.28908555]
 [0.38053097]
 [0.25958702]
 [0.09292035]
 [0.14452802]
 [0.31415929]
 [0.49557522]
 [0.07669617]
 [0.08554572]
 [0.09026549]
 [0.15560472]
 [0.02640118]
 [0.50958702]
 [0.13126844]
 [0.14882006]
 [0.04867257]
 [0.30678466]
 [0.08259587]
 [0.09587021]
 [0.25501475]
 [0.10619469]
 [0.07079646]
 [0.1039823 ]
 [0.10029499]
 [0.09948378]
 [0.16740413]
 [0.18657817]
 [0.1320059 ]
 [0.09587021]
 [0.23230088]
 [0.10619469]
 [0.23303835]
 [0.17920354]
 [0.07

#### Find the min-max scaled features in your training set that have a variance above 0.1 (see Section 10.1 in the Machine Learning with Python Cookbook).

In [375]:
# Next I will create a threshholder
test_thresholder = VarianceThreshold(threshold=.1)

In [381]:
# Next I will create a high variance feature matrix
x_test_feature_high_var = test_thresholder.transform(x_test.values.reshape(-1,1))
x_test_feature_high_var

array([[200624],
       [133000],
       [110000],
       [192000],
       [ 88000],
       [ 85000],
       [282922],
       [141000],
       [745000],
       [148800],
       [208900],
       [136905],
       [225000],
       [123000],
       [119200],
       [145000],
       [190000],
       [123600],
       [149350],
       [155000],
       [166000],
       [144500],
       [110000],
       [174000],
       [185000],
       [168000],
       [177500],
       [ 84500],
       [320000],
       [118500],
       [110000],
       [213000],
       [156000],
       [250000],
       [372500],
       [175000],
       [277500],
       [112500],
       [263000],
       [325000],
       [243000],
       [130000],
       [164990],
       [280000],
       [403000],
       [119000],
       [125000],
       [128200],
       [172500],
       [ 84900],
       [412500],
       [156000],
       [167900],
       [100000],
       [275000],
       [123000],
       [132000],
       [239900],
       [139000

In [382]:
# Next I will create a high variance feature matrix
y_test_feature_high_var = test_thresholder.transform(y_test.values.reshape(-1,1))
y_test_feature_high_var

array([[0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
    

### Step 17: Repeat step 7 with the high variance data.

In [383]:
# Next I will fit the X and y test variables 
clf.fit(x_test_feature_high_var,y_test_feature_high_var)

LinearRegression()

In [384]:
# Next I will predict the output by passing the x_test variable.
clf.predict(x_test_feature_high_var)

array([[0.77601034],
       [0.80493827],
       [0.81477711],
       [0.77969948],
       [0.82418819],
       [0.82547151],
       [0.74080523],
       [0.80151606],
       [0.54313934],
       [0.7981794 ],
       [0.77247007],
       [0.8032678 ],
       [0.76558287],
       [0.80921603],
       [0.81084157],
       [0.79980495],
       [0.78055503],
       [0.80895936],
       [0.79794413],
       [0.79552719],
       [0.79082166],
       [0.80001884],
       [0.81477711],
       [0.78739945],
       [0.78269391],
       [0.7899661 ],
       [0.78590223],
       [0.8256854 ],
       [0.72494415],
       [0.81114102],
       [0.81477711],
       [0.77071618],
       [0.79509942],
       [0.75488847],
       [0.70248591],
       [0.78697167],
       [0.74312463],
       [0.81370767],
       [0.74932738],
       [0.72280527],
       [0.7578829 ],
       [0.80622159],
       [0.79125371],
       [0.74205519],
       [0.68943874],
       [0.81092713],
       [0.80836047],
       [0.806

In [387]:
# Next I will make a prediction to use train the algorithm
y_pred_test = clf.predict(x_test_feature_high_var)

In [388]:
# Next I will test the accuracy of the model
clf.score(x_test_feature_high_var, y_test_feature_high_var)

0.007468608055410164

#### R2-value Metric

In [389]:
# Next I will calculate the r2 score by using my y_test data and the predicted data
from sklearn.metrics import r2_score
r2 = r2_score(y_test_feature_high_var, y_pred_test)
print('r2 score for perfect model is', r2)

r2 score for perfect model is 0.007468608055410164


#### RMSE - Root Mean Squared Error Metric

In [390]:
# Next I will calculate the RMSE, MSE, and MAE to compare how well the algorithm performs
from sklearn import metrics
print('RMSE - Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_feature_high_var, y_pred_test)))
print('MAE - Mean Absolute Error:', metrics.mean_absolute_error(y_test_feature_high_var, y_pred_test))
print('MSE - Mean Squared Error:', metrics.mean_squared_error(y_test_feature_high_var, y_pred_test))

RMSE - Root Mean Squared Error: 0.4098050352448076
MAE - Mean Absolute Error: 0.33588033382399596
MSE - Mean Squared Error: 0.16794016691199798


### Step 17: Summarize your findings.

During my research I found that the variables'Alley', 'FireplaceQu', 'PoolQC', 'Fence', and 'MiscFeature' could be deleted dueto the fact that they where missing more than 40% of their values. I also learned that the OverallQual 79.10% was the most important features that was relevant to Sales Price. As I went through my data I learned that I had 49 Numerical features and 26 Categorical features. [image.png](attachment:image.png) Then I found that my R2-value was 0.0074 and 40.90% for my RMSE for my linear regression on the test set. Once I Fit and transform the training features with a PCA so that 90% of the variance I found that my features for my PCA-transformed matrix stayed at 1 for my trained data. Once I transformed but did not fit the test features with the same PCA I found that my R2-value to be 74.6%. Then when I took my original training features and apply a min-max scaler to them I found a decent size array of outputs. Then when I repeated step 7 with the high variance data I found the same R2-value was 0.0074 and 40.90% for my RMSE.